
## 1. CONFIGURAÇÃO INICIAL


In [14]:
# RAG para Documentos de Condomínio - Versão Otimizada

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from datetime import datetime

# Imports LangChain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import UnstructuredFileLoader
from langchain.prompts import PromptTemplate


# ============================================================================
# 1. CONFIGURAÇÃO INICIAL
# ============================================================================

# Configuração de modelos e diretórios
MODEL = "deepseek-chat"  # "gpt-4o-mini" | "gemini-2.0-flash-exp" | "deepseek-chat"
DB_NAME = "vector_db"

# Carregamento de variáveis de ambiente
load_dotenv(override=True)

# Função de inicialização dinâmica
def get_llm_provider(model_name: str, temperature: float = 0):
    """
    Inicializa o LLM e embeddings baseado no modelo especificado
    """
    if "gpt" in model_name:
        from langchain_openai import ChatOpenAI, OpenAIEmbeddings
        
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise ValueError("OPENAI_API_KEY não encontrada nas variáveis de ambiente")
            
        llm = ChatOpenAI(
            model=model_name, 
            temperature=temperature,
            api_key=api_key
        )
        embeddings = OpenAIEmbeddings(api_key=api_key)
        
    elif "gemini" in model_name:
        from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
        
        api_key = os.getenv('GOOGLE_API_KEY')
        if not api_key:
            raise ValueError("GOOGLE_API_KEY não encontrada nas variáveis de ambiente")
            
        llm = ChatGoogleGenerativeAI(
            model=model_name, 
            temperature=temperature,
            google_api_key=api_key
        )
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=api_key
        )
        
    elif "deepseek" in model_name:
        from langchain_openai import ChatOpenAI, OpenAIEmbeddings
        
        # DeepSeek usa API compatível com OpenAI
        deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
        if not deepseek_api_key:
            raise ValueError("DEEPSEEK_API_KEY não encontrada nas variáveis de ambiente")
            
        llm = ChatOpenAI(
            model=model_name,
            temperature=temperature,
            api_key=deepseek_api_key,
            base_url="https://api.deepseek.com"  # URL base da API DeepSeek
        )
        
        # Para embeddings, usando OpenAI (você pode configurar outro provider se preferir)
        openai_api_key = os.getenv('OPENAI_API_KEY')
        if not openai_api_key:
            raise ValueError("OPENAI_API_KEY necessária para embeddings (ou configure outro provider)")
            
        embeddings = OpenAIEmbeddings(api_key=openai_api_key)
        
    else:
        raise ValueError(f"Modelo desconhecido: {model_name}")
        
    return llm, embeddings

# Inicializa o LLM e embeddings
try:
    llm, embeddings = get_llm_provider(MODEL)
    print(f"✅ Modelo {MODEL} inicializado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao inicializar modelo {MODEL}: {e}")
    raise

✅ Modelo deepseek-chat inicializado com sucesso!


## 2. CARREGAMENTO E PROCESSAMENTO DE DOCUMENTOS

In [15]:
# ============================================================================
# 3. CARREGAMENTO DE DOCUMENTOS (ALTERNATIVO) 

def load_documents():
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../0_base_conhecimento/markdown_docs"))
    
    print(f"📁 Base de busca: {BASE_DIR}")
    
    documents = []
    all_md_files = glob.glob(os.path.join(BASE_DIR, "**", "*.md"), recursive=True)
    
    print(f"🔍 Total de arquivos .md encontrados: {len(all_md_files)}")
    for file in all_md_files[:5]:  # Exibe amostra dos arquivos
        print(f"   - {os.path.relpath(file, BASE_DIR)}")
    
    for md_file in all_md_files:
        try:
            loader = TextLoader(md_file, encoding='utf-8')
            docs = loader.load()
        except Exception as e:
            print(f"⚠️ Erro ao carregar {md_file}: {e}")
            continue
               

        for doc in docs:
            content = str(doc.page_content).strip()
            if len(content) < 50:
                print(f"   ⚠️ Ignorado (muito curto): {os.path.basename(md_file)} ({len(content)} chars)")
                continue

            # 🔍 Extrai o tipo de documento da subpasta
            rel_path = os.path.relpath(md_file, BASE_DIR)
            path_parts = rel_path.split(os.sep)
            doc_type = path_parts[0] if len(path_parts) > 1 else "desconhecido"

            new_doc = Document(
                page_content=content,
                metadata={
                    "source": doc.metadata.get("source", md_file),
                    "filename": os.path.basename(md_file),
                    "rel_path": rel_path,
                    "content_length": len(content),
                    "doc_type": doc_type  # ✅ Aqui adiciona dinamicamente
                }
            )
            documents.append(new_doc)

    
    print(f"✅ Documentos carregados: {len(documents)}")
    
    if documents:
        total_chars = sum(doc.metadata["content_length"] for doc in documents)
        print(f"📊 Total de caracteres: {total_chars:,}")
        print(f"📊 Média por documento: {total_chars // len(documents):,} chars")
    
    return documents


In [16]:
load_documents()

📁 Base de busca: c:\Users\willi\Documents\Projetos\rag_condominio\0_base_conhecimento\markdown_docs
🔍 Total de arquivos .md encontrados: 31
   - docs_condominio\assembleias\065b78d0-c521-4ff9-9932-3825b4b82437.md
   - docs_condominio\assembleias\4ef8fafa-78b6-4f4e-8c81-6ca0132fe3b4.md
   - docs_condominio\assembleias\5f64b310-fb6b-412e-b295-96c52a2a1cf4.md
   - docs_condominio\assembleias\5f6bce90-49ed-4d61-858a-fba2f2e1bfe0.md
   - docs_condominio\assembleias\633681a6-5e09-4894-b648-7204c60baea2.md
✅ Documentos carregados: 31
📊 Total de caracteres: 450,970
📊 Média por documento: 14,547 chars


[Document(metadata={'source': 'c:\\Users\\willi\\Documents\\Projetos\\rag_condominio\\0_base_conhecimento\\markdown_docs\\docs_condominio\\assembleias\\065b78d0-c521-4ff9-9932-3825b4b82437.md', 'filename': '065b78d0-c521-4ff9-9932-3825b4b82437.md', 'rel_path': 'docs_condominio\\assembleias\\065b78d0-c521-4ff9-9932-3825b4b82437.md', 'content_length': 10152, 'doc_type': 'docs_condominio'}, page_content='Rua Conselheiro Nabuco; 151 Casa Amarela; RecifelPE CNPJ: 00.286.726/0001-32\n\n## ATA DA ASSEMBLEIA GERAL\\_ORDINÁRIA REALIZADA EM 15.02.2024\n\nAo décimo quinto dia do mês de fevereiro de dois mil e vinte e quatro; em 29 convocação; isto é, às 2Ohs, no salão de festas do Condomínio, reuniram-se os condôminos das unidades: 0201, 0202, 0301, 0302, 0502, 0902, 1001, 1102, 1302, 1501 atendendo a0 Edital de Convocação enviado a todas as unidades por e-mail, para deliberarem sobre as seguintes ordens do dia: IIEM 1 Aprovação da prestação de contas 2 Aprovação\\_da\\_previsão orçamentária para

In [17]:
def create_chunks(documents):
    """Cria chunks otimizados para documentos de condomínio"""
    
    if not documents:
        print("❌ Nenhum documento para processar")
        return []
    
    # Splitter otimizado para documentos legais/administrativos
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,  # Menor para melhor precisão
        chunk_overlap=150,  # Overlap maior para manter contexto
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
        length_function=len
    )
    
    chunks = text_splitter.split_documents(documents)
    
    # Adiciona informações extras aos metadados dos chunks
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk_id"] = i
        chunk.metadata["chunk_size"] = len(chunk.page_content)
    
    print(f"🔗 Total de chunks criados: {len(chunks)}")
    print(f"📋 Tipos de documento encontrados: {set(doc.metadata['doc_type'] for doc in documents)}")
    
    return chunks

## 3. CRIAÇÃO DO VECTOR STORE

In [18]:
# ============================================================================
# 3. CRIAÇÃO DO VECTOR STORE
# ============================================================================

def create_vectorstore(chunks, force_recreate=False):
    """Cria vectorstore otimizado com opção de incremental"""
    
    if not chunks:
        print("❌ Nenhum chunk fornecido para o vectorstore")
        return None
    
    try:
        # Embeddings com modelo mais recente
        print("🔧 Criando embeddings...")
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        
        # Verifica se já existe um vectorstore
        vectorstore_exists = os.path.exists(DB_NAME)
        
        if vectorstore_exists and not force_recreate:
            print(f"📂 Vectorstore existente encontrado: {DB_NAME}")
            
            # Carrega vectorstore existente
            vectorstore = Chroma(
                persist_directory=DB_NAME,
                embedding_function=embeddings,
                collection_name="condo_docs"
            )
            
            existing_count = vectorstore._collection.count()
            print(f"📊 Documentos existentes: {existing_count}")
            
            # Verifica quais documentos são novos
            existing_sources = set()
            try:
                # Pega metadados existentes para comparar
                existing_data = vectorstore._collection.get(include=["metadatas"])
                existing_sources = {
                    meta.get("source", "") + "_" + meta.get("filename", "") 
                    for meta in existing_data["metadatas"]
                }
            except:
                print("⚠️ Não foi possível verificar documentos existentes")
            
            # Filtra apenas chunks novos
            new_chunks = []
            for chunk in chunks:
                chunk_id = chunk.metadata.get("source", "") + "_" + chunk.metadata.get("filename", "")
                if chunk_id not in existing_sources:
                    new_chunks.append(chunk)
            
            if new_chunks:
                print(f"➕ Adicionando {len(new_chunks)} novos chunks...")
                vectorstore.add_documents(new_chunks)
                new_count = vectorstore._collection.count()
                print(f"✅ Total após adição: {new_count} documentos (+{new_count - existing_count})")
            else:
                print("ℹ️  Nenhum documento novo encontrado")
            
        else:
            if vectorstore_exists:
                # Remove database anterior se forçando recriação
                import shutil
                shutil.rmtree(DB_NAME)
                print(f"🗑️ Database anterior removido: {DB_NAME}")
            
            # Testa embeddings com um chunk pequeno primeiro
            print("🧪 Testando embeddings...")
            test_embedding = embeddings.embed_query("teste")
            print(f"✅ Embeddings funcionando - dimensões: {len(test_embedding)}")
            
            # Cria novo vectorstore
            print(f"💾 Criando novo vectorstore com {len(chunks)} chunks...")
            vectorstore = Chroma.from_documents(
                documents=chunks,
                embedding=embeddings,
                persist_directory=DB_NAME,
                collection_name="condo_docs"
            )
            
            count = vectorstore._collection.count()
            print(f"✅ Vectorstore criado com {count:,} documentos")
        
        # Teste básico de busca
        test_results = vectorstore.similarity_search("assembleia", k=1)
        print(f"🔍 Teste de busca: {len(test_results)} resultados")
        
        return vectorstore
        
    except Exception as e:
        print(f"❌ ERRO na criação do vectorstore: {e}")
        import traceback
        traceback.print_exc()
        return None

## 4. CONFIGURAÇÃO DO CHAT RAG

In [19]:
# ============================================================================
# 4. CONFIGURAÇÃO DO CHAT RAG
# ============================================================================

def setup_rag_chain(vectorstore, llm): # Added 'llm' as a parameter to be passed in
    """Configura a cadeia RAG com prompt customizado e data atual."""
    # Adiciona a data atual como variável para o prompt
    current_date = datetime.now().strftime("%d/%m/%Y - %A")

    # Prompt customizado para documentos de condomínio
    custom_prompt = PromptTemplate(
        template="""
        Você é um assistente especializado em documentos de condomínio. Use as informações do contexto para responder à pergunta de forma clara e precisa.

        **Data atual:** {current_date}

        **Contexto dos documentos:**
        {context}

        **Histórico da conversa:**
        {chat_history}

        **Pergunta:** {question}

        **Instruções:**
        1. Para perguntas sobre documentos do condomínio: responda baseado *apenas* nas informações fornecidas no contexto.
        2. Para perguntas gerais (como "que horas são", "qual a data de hoje", "quem é o presidente do Brasil"): você *pode* usar seu conhecimento geral para responder.
        3. Se a informação solicitada *não* estiver disponível nos documentos do condomínio, diga claramente que "A informação solicitada não está disponível nos documentos do condomínio."
        4. Sempre que possível, **cite as seções, artigos ou parágrafos relevantes** dos documentos que fundamentam sua resposta.
        5. Seja objetivo, direto e use linguagem clara e concisa.
        6. Ao responder sobre valores, datas, prazos ou procedimentos, seja o mais específico possível.

        **Resposta:**""",
        input_variables=["context", "chat_history", "question", "current_date"] # Added current_date here
    )
    
    # Memória
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
        output_key='answer'
    )
    
    # Retriever com configurações otimizadas
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Número de chunks a recuperar
    )
    
    # Chain
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={"prompt": custom_prompt} # Pass the custom prompt here
    )
    
    return conversation_chain

## 8. FUNÇÕES UTILITÁRIAS PARA GERENCIAR DOCUMENTOS

In [22]:
# ============================================================================
# INTERFACE GRADIO COM GERENCIAMENTO DE DOCUMENTOS
# ============================================================================

import gradio as gr
import os
import shutil
from datetime import datetime

# ============================================================================
# FUNÇÕES UTILITÁRIAS (do arquivo fornecido)
# ============================================================================

def add_new_document(file_path, doc_type="novo_documento"):
    """Adiciona um único documento novo ao vectorstore existente"""
    try:
        from langchain.document_loaders import TextLoader, UnstructuredFileLoader
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain.embeddings import OpenAIEmbeddings
        from langchain.vectorstores import Chroma
        
        # Carrega o documento
        if file_path.endswith('.txt'):
            loader = TextLoader(file_path, encoding='utf-8')
        elif file_path.endswith('.docx'):
            loader = UnstructuredFileLoader(file_path)
        elif file_path.endswith('.md'):
            loader = TextLoader(file_path, encoding='utf-8')
        else:
            return f"❌ Formato não suportado: {file_path}"
        
        docs = loader.load()
        
        # Processa o documento
        for doc in docs:
            content = str(doc.page_content).strip()
            if len(content) < 50:
                return f"⚠️ Documento muito pequeno: {len(content)} chars"
            
            doc.metadata.update({
                "doc_type": doc_type,
                "filename": os.path.basename(file_path),
                "content_length": len(content)
            })
        
        # Cria chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=150,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            length_function=len
        )
        chunks = text_splitter.split_documents(docs)
        
        # Adiciona ao vectorstore existente
        DB_NAME = "chroma_db"  # Definir se não existir
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        vectorstore = Chroma(
            persist_directory=DB_NAME,
            embedding_function=embeddings,
            collection_name="condo_docs"
        )
        
        old_count = vectorstore._collection.count()
        vectorstore.add_documents(chunks)
        new_count = vectorstore._collection.count()
        
        result = f"✅ Documento adicionado: {os.path.basename(file_path)}\n"
        result += f"📊 Chunks criados: {len(chunks)}\n"
        result += f"📈 Total no banco: {old_count} → {new_count} (+{new_count - old_count})"
        
        return result
        
    except Exception as e:
        return f"❌ Erro ao adicionar documento: {e}"

def list_documents_in_vectorstore():
    """Lista todos os documentos no vectorstore"""
    try:
        from langchain.embeddings import OpenAIEmbeddings
        from langchain.vectorstores import Chroma
        
        DB_NAME = "chroma_db"
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        vectorstore = Chroma(
            persist_directory=DB_NAME,
            embedding_function=embeddings,
            collection_name="condo_docs"
        )
        
        # Pega todos os metadados
        data = vectorstore._collection.get(include=["metadatas"])
        
        # Agrupa por documento
        docs_info = {}
        for meta in data["metadatas"]:
            doc_key = meta.get("filename", "unknown")
            doc_type = meta.get("doc_type", "unknown")
            
            if doc_key not in docs_info:
                docs_info[doc_key] = {
                    "type": doc_type, 
                    "chunks": 0,
                    "source": meta.get("source", "unknown")
                }
            docs_info[doc_key]["chunks"] += 1
        
        result = f"📋 Documentos no vectorstore ({len(docs_info)} arquivos):\n\n"
        for filename, info in docs_info.items():
            result += f"📄 {filename} ({info['type']}) - {info['chunks']} chunks\n"
        
        total_chunks = sum(info["chunks"] for info in docs_info.values())
        result += f"\n📊 Total: {total_chunks} chunks"
        
        return result
        
    except Exception as e:
        return f"❌ Erro ao listar documentos: {e}"

def clear_vectorstore_safe():
    """Remove completamente o vectorstore com confirmação"""
    try:
        DB_NAME = "chroma_db"
        if os.path.exists(DB_NAME):
            shutil.rmtree(DB_NAME)
            return f"🗑️ Vectorstore removido: {DB_NAME}\n⚠️ Você precisará recriar o banco de dados!"
        else:
            return "ℹ️ Vectorstore não existe"
    except Exception as e:
        return f"❌ Erro ao remover vectorstore: {e}"

# ============================================================================
# FUNÇÕES PARA GRADIO
# ============================================================================

def gradio_add_document(file, doc_type):
    """Wrapper para adicionar documento via Gradio"""
    if file is None:
        return "❌ Nenhum arquivo selecionado"
    
    try:
        # O Gradio passa o caminho temporário do arquivo
        result = add_new_document(file.name, doc_type)
        
        # Atualizar o conversation_chain se existir
        try:
            if 'conversation_chain' in globals():
                print("🔄 Atualizando conversation_chain...")
                # Recriar o retriever com os novos documentos
                from langchain.embeddings import OpenAIEmbeddings
                from langchain.vectorstores import Chroma
                
                DB_NAME = "chroma_db"
                embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
                vectorstore = Chroma(
                    persist_directory=DB_NAME,
                    embedding_function=embeddings,
                    collection_name="condo_docs"
                )
                
                # Atualizar o retriever do chain
                if hasattr(globals()['conversation_chain'], 'retriever'):
                    globals()['conversation_chain'].retriever = vectorstore.as_retriever(
                        search_type="similarity",
                        search_kwargs={"k": 3}
                    )
                
                result += "\n🔄 Sistema atualizado com novo documento!"
        except Exception as e:
            result += f"\n⚠️ Aviso: Erro ao atualizar sistema: {e}"
        
        return result
        
    except Exception as e:
        return f"❌ Erro: {e}"

def gradio_list_documents():
    """Wrapper para listar documentos via Gradio"""
    return list_documents_in_vectorstore()

def gradio_clear_vectorstore():
    """Wrapper para limpar vectorstore via Gradio"""
    return clear_vectorstore_safe()

def gradio_chat_function(message, history):
    """Função de chat integrada"""
    if 'adaptive_chat_function' in globals():
        return globals()['adaptive_chat_function'](message, history)
    else:
        return "❌ Sistema de chat não inicializado. Execute primeiro o setup do RAG."

def gradio_system_status():
    """Verifica status do sistema"""
    status = "🔍 STATUS DO SISTEMA\n" + "="*30 + "\n\n"
    
    # Verificar componentes
    components = [
        ('conversation_chain', 'Chat RAG'),
        ('vectorstore', 'Base de Dados'),
        ('llm', 'Modelo de Linguagem')
    ]
    
    for var_name, desc in components:
        if var_name in globals():
            status += f"✅ {desc}: OK\n"
        else:
            status += f"❌ {desc}: Não encontrado\n"
    
    # Verificar arquivos
    DB_NAME = "chroma_db"
    if os.path.exists(DB_NAME):
        status += f"✅ Database: {DB_NAME} existe\n"
    else:
        status += f"❌ Database: {DB_NAME} não encontrado\n"
    
    # Contar documentos
    try:
        docs_info = list_documents_in_vectorstore()
        if "❌" not in docs_info:
            lines = docs_info.split('\n')
            total_line = [line for line in lines if "Total:" in line]
            if total_line:
                status += f"📊 {total_line[0]}\n"
    except:
        status += "⚠️ Não foi possível contar documentos\n"
    
    status += f"\n🕒 Verificado em: {datetime.now().strftime('%H:%M:%S')}"
    
    return status

# ============================================================================
# INTERFACE GRADIO PRINCIPAL
# ============================================================================

def create_gradio_interface():
    """Cria interface Gradio com múltiplas abas"""
    
    with gr.Blocks(
        title="🏢 Sistema RAG - Assistente de Condomínio",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        .tab-nav {
            font-weight: bold;
        }
        """
    ) as interface:
        
        gr.Markdown("""
        # 🏢 Sistema RAG - Assistente de Condomínio
        **Gerencie documentos e converse com seu assistente inteligente**
        """)
        
        with gr.Tabs():
            
            # ABA 1: CHAT
            with gr.Tab("💬 Chat", elem_classes=["tab-nav"]):
                gr.Markdown("### Converse com o assistente sobre os documentos do condomínio")
                
                chatbot = gr.ChatInterface(
                    fn=gradio_chat_function,
                    examples=[
                        "Quando foi a última assembleia?",
                        "Quais contratos estão vigentes?",
                        "Qual o valor da taxa condominial?",
                        "Resumo dos principais pontos das atas",
                        "Quem são os síndicos atuais?"
                    ],
                    title=None
                )
            
            # ABA 2: GERENCIAR DOCUMENTOS
            with gr.Tab("📁 Gerenciar Documentos", elem_classes=["tab-nav"]):
                gr.Markdown("### Adicione, liste ou remova documentos do sistema")
                
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("#### ➕ Adicionar Documento")
                        
                        file_input = gr.File(
                            label="Selecione arquivo (.txt ou .docx ou .md)",
                            file_types=[".txt", ".docx", ".md"],
                        )
                        
                        doc_type_input = gr.Dropdown(
                            choices=[
                                "ata_assembleia",
                                "contrato",
                                "regulamento",
                                "correspondencia",
                                "financeiro",
                                "manutencao",
                                "outro"
                            ],
                            value="outro",
                            label="Tipo do documento"
                        )
                        
                        add_btn = gr.Button("📥 Adicionar Documento", variant="primary")
                        add_result = gr.Textbox(
                            label="Resultado",
                            lines=5,
                            interactive=False
                        )
                    
                    with gr.Column(scale=1):
                        gr.Markdown("#### 📋 Listar Documentos")
                        
                        list_btn = gr.Button("📄 Listar Todos os Documentos")
                        list_result = gr.Textbox(
                            label="Documentos no Sistema",
                            lines=10,
                            interactive=False
                        )
                
                gr.Markdown("---")
                
                with gr.Row():
                    gr.Markdown("#### ⚠️ Zona Perigosa")
                    
                with gr.Row():
                    clear_btn = gr.Button("🗑️ Limpar Banco de Dados", variant="stop")
                    clear_result = gr.Textbox(
                        label="Resultado da Limpeza",
                        lines=3,
                        interactive=False
                    )
                
                # Eventos
                add_btn.click(
                    gradio_add_document,
                    inputs=[file_input, doc_type_input],
                    outputs=add_result
                )
                
                list_btn.click(
                    gradio_list_documents,
                    outputs=list_result
                )
                
                clear_btn.click(
                    gradio_clear_vectorstore,
                    outputs=clear_result
                )
            
            # ABA 3: STATUS DO SISTEMA
            with gr.Tab("⚙️ Status", elem_classes=["tab-nav"]):
                gr.Markdown("### Verificar status dos componentes do sistema")
                
                status_btn = gr.Button("🔍 Verificar Status", variant="secondary")
                status_result = gr.Textbox(
                    label="Status do Sistema",
                    lines=15,
                    interactive=False
                )
                
                status_btn.click(
                    gradio_system_status,
                    outputs=status_result
                )
                
                # Auto-verificar status ao carregar
                interface.load(
                    gradio_system_status,
                    outputs=status_result
                )
            
            # ABA 4: AJUDA
            with gr.Tab("❓ Ajuda", elem_classes=["tab-nav"]):
                gr.Markdown("""
                ### 📖 Como usar o sistema
                
                #### 💬 **Chat**
                - Faça perguntas sobre os documentos do condomínio
                - O sistema buscará informações relevantes para responder
                - Use perguntas específicas para melhores resultados
                
                #### 📁 **Gerenciar Documentos**
                - **Adicionar**: Envie arquivos .txt ou .docx ou .md
                - **Listar**: Veja todos os documentos indexados
                - **Limpar**: Remove todo o banco de dados (cuidado!)
                
                #### ⚙️ **Status**
                - Verifica se todos os componentes estão funcionando
                - Mostra quantos documentos estão no sistema
                - Útil para diagnosticar problemas
                
                #### 🔧 **Tipos de Documento**
                - **ata_assembleia**: Atas de assembleias
                - **contrato**: Contratos de serviços
                - **regulamento**: Regulamentos internos
                - **correspondencia**: Cartas e ofícios
                - **financeiro**: Demonstrativos financeiros
                - **manutencao**: Relatórios de manutenção
                - **outro**: Outros documentos
                
                #### ⚠️ **Dicas Importantes**
                - Depois de adicionar documentos, teste o chat
                - Use nomes descritivos nos arquivos
                - Docx podem demorar mais para processar
                - Backup seus documentos importantes antes de limpar
                """)
        
        gr.Markdown("""
        ---
        *Sistema RAG para Condomínios - Versão com Gerenciamento de Documentos*
        """)
    
    return interface

def launch_complete_gradio():
    """Lança a interface completa do Gradio"""
    try:
        print("🚀 Iniciando interface completa...")
        
        interface = create_gradio_interface()
        
        print("🌐 Lançando em http://127.0.0.1")
        interface.launch(
            inbrowser=True,
            server_name="127.0.0.1",
            server_port=7864,
            share=False
        )
        
    except Exception as e:
        print(f"❌ Erro ao lançar interface: {e}")


In [23]:

# ============================================================================
# EXECUTAR INTERFACE
# ============================================================================

if __name__ == "__main__":
    launch_complete_gradio()

# Para executar no notebook/script:
print("\n" + "="*60)
print("🚀 INTERFACE COMPLETA DISPONÍVEL!")
print("Execute: launch_complete_gradio()")
print("="*60)

🚀 Iniciando interface completa...
🌐 Lançando em http://127.0.0.1
* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.



🚀 INTERFACE COMPLETA DISPONÍVEL!
Execute: launch_complete_gradio()
